# COBRApy

COBRApy is a package for constraint-based modeling of biological networks written in Python.

This tool allows loading and inspecting Genome-Scale Metabolic (GEM) models written in the Systems Biology Markup Language (SBML) format.

Using COBRApy, one can analyze the following model contents:
1. Reactions
2. Metabolites
3. Genes
4. Exchange reactions (Environmental Conditions)

COBRApy allows manipulating the contents of a GEM model. For instance, one can edit reactions' flux bounds, knock out a metabolic gene, or change environmental conditions.

COBRApy contains flux analysis methods to simulate an organism's phenotypic behavior. These include Flux Balance Analysis (FBA), Parsimonious FBA, or Flux Variability Analysis (FVA).

The simulation of gene and reaction deletions for a given GEM model is a straightforward process. One can simulate single or double knockouts using one of the flux analysis methods mentioned above.

## Installation


### Requirements
The following requirements are needed to use COBRApy:
- Python 3.6 or higher
- pip must be installed
- GLPK is the default solver, but CPLEX is preferred


### How to install COBRApy?
```
pip install cobra
```

# Exercise 2 - Flux analysis & Constraint-based modeling

## Working with a GEM model

COBRApy can be used to read a GEM model in SBML format file.
For this practical session, we will be using the following model:
- _textbook_ model which contains the metabolic network presented in the _exercise 1_ -> file: ../data/textbook.xml

The reactions, metabolites, and genes encoded into a SBML format file can be parsed by COBRApy. These contents are loaded into Python objects simple to use, namely `cobra.Reaction`, `cobra.Metabolite`, and `cobra.Gene`

The model itself will be available as an `cobra.Model` object containing all these attributes.

In [1]:
# importing cobra
import cobra

In [2]:
# Loading a model
model_path = '../data/textbook.xml'
model = cobra.io.read_sbml_model(model_path)

model

Name,textbook
Memory address,16be51ec408
Number of metabolites,9
Number of reactions,12
Number of genes,0
Number of groups,0
Objective expression,1.0*R4 - 1.0*R4_reverse_8717c
Compartments,c


### Linear problem
The degrees of freedom can be calculated by subtracting the number of variables by the number of linear mass equations. That is, one can determine the degree of under-determination of this model by subtracting the number of reactions by the number of metabolites.

In [3]:
# calculating the linear problem degrees of freedom
degrees = len(model.reactions) - len(model.metabolites)
degrees

3

## Environmental Conditions


The set of measured exchange fluxes can be used to define the environmental conditions of our _textbook_ model. Thus, we can use the measurements obtained to turn our under-determined system into a determined system ready to be solved.

In [4]:
# create the environmental conditions with our measurements
environmental_conditions = {
    'R1': (-1, 0),
    'R2': (0.5, 1000),
    'R3': (0.5, 1000),
}
environmental_conditions

{'R1': (-1, 0), 'R2': (0.5, 1000), 'R3': (0.5, 1000)}

In [5]:
# set the environmental conditions in the textbook model
for reaction_id, bound in environmental_conditions.items():
    model.reactions.get_by_id(reaction_id).bounds = bound

### Flux analysis with FBA

COBRApy allows performing Flux Balance Analysis (FBA) in GEM models.

To perform a simulation using this method, you should first define an objective function. This can be a reaction or an exchange, which will be maximized or minimized. 

By default, the _textbook_ model objective function is reaction 4, R4. 
This can be thought of as the maximization of the ethanol production.

In [6]:
model.objective.expression

1.0*R4 - 1.0*R4_reverse_8717c

#### Flux Balance Analysis (FBA)

FBA simulations can be performed using `model.optimize()`. One can easily see the flux distribution as a result.

In [7]:
flux_distribution = model.optimize()
flux_distribution

,fluxes,reduced_costs
V1,2.00,0.0
V2,0.50,0.0
V3,1.00,0.0
V4,0.50,0.0
V5,0.25,0.0
...,...,...
R2,0.50,-2.0
R3,0.50,-1.0
R4,1.25,0.0
R5,0.25,0.0


### Pathway Visualization

COBRApy does not include any pathway visualization method. Nevertheless, independent tools that work with cobra models can be used for that goal. One such tool is named Escher.

Escher allows the visualization of metabolic pathway maps. Escher maps can be visualized using the `escher.Build()` function, which requires the model, the metabolic pathway map, and an FBA solution.

To obtain the metabolic map for the model in question, Escher can be run as follows:

In [8]:
# import escher
import escher

In [9]:
builder = escher.Builder(map_json='../data/textbook_map.json', model=model, reaction_data=flux_distribution.fluxes)
builder

Builder(reaction_data={'V1': 2.0, 'V2': 0.5, 'V3': 1.0, 'V4': 0.5, 'V5': 0.25, 'V6': 1.25, 'R1': -1.0, 'R2': 0…